# Object oriented error handling
- Errors are defined by classes
- an error object is instantiated at error time
    - the class used tells you something about the problem
    - the error object may include specific information about the problem, like a file path that doesn't exist
- errors can be raised/thrown either by your code, or Python itself
- errors may be caught/handeled by your code
- Java - compiler DEMANDS you handle all kinds of potential errors
- Python - hey whatever...
- but, if you get an error, and there's no handler, you CRASH in the debugger



# some errors are raised by Python
- [List of builtin errors](https://docs.python.org/3/library/exceptions.html#exception-hierarchy)
- error types form a hierarchy(formed by using single inheritance)

In [1]:
a = 0
b = 1
b/a

ZeroDivisionError: division by zero

# can explicitly 'raise'(or throw) an error 
- 'raise' statement takes an error object, and invokes the Python error handling system on it

In [2]:
def picky(n):
    if n == 3 or n == 6:
        # standard way
        raise ValueError(f'picky is unhappy - bad value n={n}')
    return n+1

In [3]:
# no problem here

picky(1)

2

In [4]:
# but this will bomb - error was thrown but not caught
# so program stops in the debugger - usually very undesirable

picky(6)

ValueError: picky is unhappy - bad value n=6

In [5]:
def picky2(n):
    if n == 3 or n == 6:
        # instantiate error object first
        # then put extra error info on the error object
        ve = ValueError(f'picky is unhappy - bad value n={n}')
        ve.badn = n
        raise ve
    return n+1


# catch/handle errors with try/except


In [6]:
try:
    picky2(3)
except Exception as e:
    # catch the error
    # e is the error object from the raise statement
    # e.badn has the bad int value
    print(e, e.badn)

# execution continues normally

'now we are here' 

picky is unhappy - bad value n=3 3


'now we are here'

In [7]:
# input function reads a line of text from terminal 

input()

bob


'bob'

In [8]:
# insist on reading an int

def getn():
    while True:
        try:
            s = input('gimme an int!!')
            j = int(s)
            return j
        except Exception as e:
            print(e)

getn()
        

gimme an int!!bob
invalid literal for int() with base 10: 'bob'
gimme an int!!df
invalid literal for int() with base 10: 'df'
gimme an int!!3


3

# 'finally' clause is always executed, with or without an error
- gives you a chance to do something before
you lose control

In [9]:
# no error, but finally clause is executed

try:
    picky(1)
except Exception:
    print('got error')
finally:
    print('finally clause')
'now we are here'

finally clause


'now we are here'

In [10]:
# error thrown, but finally clause still executed

try:
    picky(3)
except Exception:
    print('got error')
finally:
    print('finally clause')
'now we are here'

got error
finally clause


'now we are here'

# stack frame, call stack
- when a function is called, a 'stack frame' is pushed onto the call stack
    - stack frames hold various bindings for the frame
- when a function exits, its stack frame is popped off the call stack, and discarded

# stack frames display
- if no error handler is found, the top
level will display a 'stack trace', and your entire computation is terminated
- shows context of error
- very useful for debugging
- note: the most recent call frame is printed last
- try code below with [Python Tutor](http://pythontutor.com/visualize.html#mode=edit)

In [11]:
# simple recursion

def fact(n):
    if n == 1:
        return 1
    else:
        return n * fact(n-1)
    
fact(3)

6

In [12]:
def A(n):
    z = 1
    B(1)

def B(n):
    z = "dog"
    C(2)

def C(n):
    z = []
    D(3)
    
def D(n):
    z = {}
    raise Exception("stack frames will be displayed")

A(10)

Exception: stack frames will be displayed

# can define a custom error class
- usually inherit from 'Exception'

# breaking out of nested loops                    

In [13]:
# note use of 'pass'

class breakNested(Exception):
    pass

def bn(bval):
    try:
        for x in range(3):
            print('x',x)
            for y in range(3):
                print('y',y)
                if y == bval:
                    raise breakNested
    except breakNested:
        print('broke out of nested loop')
    print('function body continues...')
                       

# if we design our own Exception class, we can define init args for the class, and print methods (repr, str)

In [14]:
import random

class missedSecret(Exception):
    def __init__(self, secret, guess):
        self.secret = secret
        self.guess = guess
    
    def __repr__(self):
        "control how error prints"
        return f'secret was {self.secret}, guess was {self.guess}!'
    def __str__(self):
        # Exception class doesn't call __repr__() to compute __str__()
        return self.__repr__()
    
def guessSecret(guess):
    secret = 1
    if guess != secret:
        raise missedSecret(secret, guess)
    print(f'guess of {guess} is correct!')
    

In [15]:
# will loop until guess fails
try:
    for j in range(3):
        guess = random.randint(0, 2)
        guessSecret(guess)
except missedSecret as ms:
    # ms will be bound to the error object
    print(ms)

guess of 1 is correct!
secret was 1, guess was 2!


# when an error is raised, Python will search the call stack for an error handler
- Python checks the current stack frame for a handler, then checks each
older frame in turn
- if no handler is found, the debugger is entered, the error is printed, and your program dies
- try code below with  [Python Tutor](http://pythontutor.com/visualize.html#mode=edit)

In [16]:
# no error

def foo():
    try:
        bar()
    except Exception as e:
        print('caught in foo:', e)
    print('here i am in foo')
    
def bar():
    a,b = 0,1
    c = a + b
    print('here i am in bar')

foo()

here i am in bar
here i am in foo


In [17]:
# no error handler in 'bar', so look in caller, 'foo',
# which does have one.

def foo():
    try:
        bar()
    except Exception as e:
        print('caught in foo:', e)
    print('here i am in foo')
    
def bar():
    a,b = 0,1
    # division by zero error raised here
    b/a
    # never gets here
    print('here i am in bar')

foo()

caught in foo: division by zero
here i am in foo


In [18]:
# 'bar' has a handler, 
# so error is caught there

def foo():
    try:
        bar()
    except Exception as e:
        print('caught in foo:', e)
    
def bar():
    a,b = 0,1
    try:
        b/a
    except ZeroDivisionError as e:
        print('caught in bar:', e)

foo()

caught in bar: division by zero


In [19]:
# both 'foo' and 'bar' have handlers, but
# neither is the right type, so the error
# is NOT caught

def foo():
    try:
        bar()
    except FileNotFoundError as e:
        print('caught in foo:', e)
    
def bar():
    a,b = 0,1
    try:
        b/a
    except ValueError as e:
        print('caught in bar', e)

foo()

ZeroDivisionError: division by zero

# Complex error example
- you can ignore this if you wish

In [20]:
def tc(b, f):
    if b:
        try:
            return tc2(f)
        # usually a good idea to catch
        # Exception at top level
        except Exception as e:
            print('tc: caught {}'.format(e))
    else:
        try:
            return tc2(f)
        except OSError as os:
            print('tc: caught {}'.format(os))  

def tc2(f):
    try:
        return tc3(f)
    # can catch any number of error types
    # in a single try
    except FileNotFoundError as fe:
        print('tc2: caught {}'.format(fe))
    except MemoryError as me:
        print('tc2: caught {}'.format(me))
    
def tc3(f):
    try:
        return f()
    except ArithmeticError as ae:
        print('tc3: caught {}'.format(ae))  
    
def noproblem():
    a = 5/4
    return a
    
def dbz2():
    try: 
        c = 1/0
        return c
    except ZeroDivisionError as z:
        print('dbz: caught {}'.format(z))
        
def dbz3():
    a = 1/0
    return a
    
def si():
    raise StopIteration
    
def fnf():
    # can put useful information about 
    # the error into the error object
    raise FileNotFoundError('/tmp/foo')

def me():
    raise MemoryError

In [21]:
# should run fine, and not generate an error

tc(1, noproblem)

1.25

In [22]:
#  caught the error it generated, inside dbz2

tc(1, dbz2)

dbz: caught division by zero


In [23]:
# no handler in dbz3
# caught by handler in tc3

tc(1, dbz3)

tc3: caught division by zero


In [24]:
# caught in 'tc' Exception handler

tc(1, si)

tc: caught 


In [25]:
# no handler on the call stack
# top level prints stack trace, 
# and your program crashes

tc(0, si)

StopIteration: 

In [26]:
int('234')

234

In [27]:
int(234.324)

234

In [28]:
int('2343.34')

ValueError: invalid literal for int() with base 10: '2343.34'